In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-q7l0pswm
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-q7l0pswm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 121 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 40.9 MB/s 
     |████████████████████████████████| 1.7 MB 55.5 MB/s 
     |████████████████████████████████| 140 kB 53.3 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 180 kB 65.8 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=425154 sha256=8930d0a74fcd79c233690a03f0b3f76326d3830b8863cd39188d6f17b

In [2]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import SplitOmniglot
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive

In [6]:
from torchvision import transforms

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
height = 32
n_classes = 964
batch_size = 8
epochs = 5

benchmark = SplitOmniglot(n_experiences=4, 
                          seed=1, 
                          train_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]), 
                          eval_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]))

model = SimpleMLP(num_classes=n_classes, input_size=height*height)

optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=batch_size, train_epochs=epochs,
    eval_mb_size=batch_size, device=device)

print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(benchmark.test_stream))

Files already downloaded and verified
Files already downloaded and verified
Starting experiment...
Start of experience:  0
Current Classes:  [2, 8, 520, 10, 529, 531, 534, 28, 541, 542, 33, 35, 548, 550, 40, 554, 556, 47, 559, 561, 564, 567, 568, 58, 571, 573, 62, 574, 64, 580, 70, 72, 74, 589, 78, 590, 592, 593, 594, 595, 598, 87, 600, 90, 93, 607, 97, 100, 102, 614, 617, 107, 623, 626, 115, 628, 122, 123, 636, 126, 638, 129, 647, 139, 651, 653, 143, 144, 145, 146, 655, 148, 656, 657, 659, 152, 664, 668, 157, 161, 163, 676, 166, 679, 168, 681, 171, 685, 174, 686, 693, 696, 698, 187, 699, 191, 705, 706, 195, 196, 707, 708, 199, 200, 710, 202, 716, 721, 211, 212, 213, 728, 729, 218, 733, 223, 738, 739, 228, 229, 233, 241, 756, 761, 762, 763, 252, 253, 764, 255, 766, 770, 774, 775, 264, 778, 267, 269, 271, 784, 786, 789, 791, 283, 284, 796, 287, 289, 801, 804, 805, 295, 809, 812, 813, 308, 309, 310, 821, 312, 827, 829, 318, 833, 322, 835, 325, 837, 327, 840, 841, 330, 331, 847, 337, 849,